In [7]:
import pandas as pd
import os
os.getcwd()

'/home/rayane/capstone-project/test/code'

In [13]:
#replace NaN by unknown in the column "Borough"
def get_AMR_df()->pd.DataFrame:
    df = pd.read_excel("../../data/SIR_readings.xlsx", sheet_name='Consolidated_SIRs')
    #set the genome_id col as rownames:
    df.set_index('genome_id', inplace=True)

    return df

df=get_AMR_df()
df

,species,amikacin,amoxicillin,amoxicillin_clavulanic_acid,ampicillin,ampicillin_sulbactam,azithromycin,aztreonam,benzylpenicillin,cefalothin,...,sulfisoxazole,teicoplanin,telithromycin,tetracycline,ticarcillin_clavulanic_acid,tigecycline,tobramycin,trimethoprim,trimethoprim_sulphamethoxazole,vancomycin
genome_id,,,,,,,,,,,,,,,,,,,,,
470.80140,Acinetobacter baumannii,resistant,NaN,NaN,resistant,resistant,NaN,resistant,NaN,NaN,...,NaN,NaN,NaN,resistant,NaN,NaN,susceptible,NaN,resistant,NaN
470.62120,Acinetobacter baumannii,resistant,NaN,NaN,NaN,resistant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intermediate,NaN
470.62130,Acinetobacter baumannii,resistant,NaN,NaN,NaN,resistant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,resistant,NaN
470.62140,Acinetobacter baumannii,resistant,NaN,NaN,NaN,resistant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intermediate,NaN
470.62150,Acinetobacter baumannii,resistant,NaN,NaN,NaN,resistant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,resistant,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313.10321,Streptococcus pneumoniae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,resistant,NaN
1313.10320,Streptococcus pneumoniae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,susceptible,NaN
1313.10319,Streptococcus pneumoniae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,resistant,NaN


In [16]:
species=df['species'].unique()
drugs=list(df.columns[1:])
for s in species:
    for d in drugs:
        sp_df=df[df['species']==s] #filtering a new df for only this species
        sp_df=sp_df[[d]]
        print(sp_df)

              amikacin
genome_id             
470.8014     resistant
470.6212     resistant
470.6213     resistant
470.6214     resistant
470.6215     resistant
...                ...
470.7806   susceptible
470.7805   susceptible
470.7804     resistant
470.7809   susceptible
470.7803     resistant

[1328 rows x 1 columns]


KeyError: "None of [Index(['amoxicillin'], dtype='object')] are in the [columns]"